
<p><img alt="udeA logo" height="120px" src="https://github.com/freddyduitama/images/blob/master/logo.png?raw=true" align="left" hspace="10px" vspace="0px" style="width:107px;height:152px;"></p>

# <center> <font color='0B5345'>Homework. Lecture 2.2:</font> </center>

## TASK: T1
<p justify="align">
<font color='0B5345' ><b>Step 1:</b></font> Building a Bloom filter.</font>
<ul align="justify">
<li>Suppose a set $S = \{s_1, s_2,...,s_m\}$ of size $m$. The input stream is a multiset taken from $S$.</li>
<li>Use <em>four-letter keys</em> from the alphabet = <em>{A,B,C,D,E,F,G,H,I,J}</em>. As result, You will have $10^4=10000$ possible keys.</li>
<li>Define the Bloom filter using a $n$-bits array $A$. Initialize $A$ using $k$ hash functions. </li>
</ul>
<center><img align="center" src="https://raw.githubusercontent.com/freddyduitama/images/master/bloom-filter.png"  height="150" width="450"></center>

- <em>You need a false positive probability =</em> $( 1 -  \frac {1}{e^{km/n}})^k \le 0.05$
- <em>The optimal number of hash functions is </em> $k = \lceil \frac{n}{m}  \ln 2 \rceil $
- A Bloom filter of $n$ bits  keeps a false positive rate  $p$ for $m$ elements when $n = - \frac{m \ ln \ p}{ (ln \ 2)^2} $
</p>
</font>

<font color='0B5345' ><b>Step 2:</b></font> Testing an arriving  element $s_i$ in stream <br>
<left><img align= "left" src="https://raw.githubusercontent.com/freddyduitama/images/master/testing-bloom-filter.png" height="150" width="450"></left>

###  I. sign in with grader

In [2]:
#you must run this code to enable the grader
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


In [3]:
# sig in. Please don't modify this code.
# init.course_id = "LOGICA_3.v1"
init.course_id = "LOGICA_3.v1"
import inspect
from local.lib.rlxmoocapi import session
session.LoginSequence(endpoint=init.endpoint, course_id=init.course_id, lab_id="L02.02", varname="student");

logging in as estiven.carmona@udea.edu.co... please wait
using course session LOGICA_3.v1::2023.2
success!! you are logged in


### II. Add your solution

# Bloom Filter
1. Define a class and its methods for building a bloom filter.

In [4]:
 !pip install bitmap

In [5]:
!pip install primesieve==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.8/264.8 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for primesieve: filename=primesieve-2.3.0-cp310-cp310-linux_x86_64.whl size=2313847 sha256=144f07928e932705b5cc299459df97681e8c486320fb53d09d9b4f4c600bdac5
  Stored in directory: /root/.cache/pip/wheels/b4/ed/73/9fa15149adddf2abd212624bf38b1be05a50fb6a39ba9f5055
Successfully built primesieve


In [7]:
def my_solution_t1(m, n):
  # define hash_table classs and its methods
  import numpy as np
  import math
  from bitmap import BitMap
  from primesieve import n_primes
  import hashlib
  import random

  class BloomFilter:

    def __init__(self, m, n)   :
        self._buckets = n_primes(1, n+1)[0]
        ### START YOUR CODE HERE ###
        self._k = round((self._buckets / m) * math.log(2))                                                                 # define the optimal number of hash functions. Use round().
        ### END YOUR CODE HERE ###

        self._table = BitMap(self._buckets)                                       # define hash table

        ### START YOUR CODE HERE ###                                              # define k hash functions in
        self._functions = [None] * self._k                                        # list to store hash functions.
        for i in range(0, self._k):
          a = random.randint(1, self._buckets - 1)
          b = random.randint(0, self._buckets - 1)
          self._functions[i] = [a, b]
        ### END  YOUR CODE HERE ###

        self._on = 0                                                              # count bit ON in bit-array
        self._m = m

    def get_parameters(self):                                                     #
        ### START YOUR CODE HERE ###
        fp_rate = ((1 - math.exp((-self._k * self._m) / self._buckets)) ** self._k) * 100                                                          # compute false positive rate.
        ### END  YOUR CODE HERE ###
        return self._m, self._buckets, self._k, fp_rate                              # Return  m, number of buckets, k and false positive rate.

    def get_functions(self):
      return  self._functions,self._buckets

    def get_hash(self, key):

      # Calculate the hash SHA256 of the string
      sha256_hash = hashlib.sha256(key.encode()).hexdigest()
      # Convert the hash to integer
      mapped_key = int(sha256_hash, 16)
      pos = []                                                                    # Variable pos is a list. It contains k elements (k positions)
      ### START YOUR CODE HERE ###                                                # compute pos from k hash functions.
      for i in range(0, self._k):
        val = (self._functions[i][0] * mapped_key + self._functions[i][1]) % self._buckets
        pos.append(val)                                                                            # Variable pos is a list. It contains k elementos (k positions)
      ### END  YOUR CODE HERE ###
      return pos, mapped_key

    def set_item(self,positions):
      for i in range (self._k):
            self._table.set(positions[i])
      self._on = self._table.count()
      return None

    def find_item(self,key):                          # using key as parameter call method hash to find the k-positions. return the list pos.
                                                    # test the positions in pos. Return FALSE whenever you find at least one position in zero. otherwise return TRUE
      pos, mapped = self.get_hash(key)
      answer = True
      ### START YOUR CODE HERE ###
      for i in pos:
        if self._table[i] == 0:
          answer = False

      ### END  YOUR CODE HERE ###
      return answer                                 # Return True or False.

    def load_factor(self):
      return self._table.count()/ self._buckets

  return BloomFilter(m,n)

In [19]:
m = 1000         # you must calculate the correct value of m
n = 700         # you must calculate the correct value of n

### III. Test your solution

In [8]:
# Compute numbers of buckets and build hash_table
my_n = input('Please enter the range of values into the universe: ')
my_m = input('Please enter the num of elements that you want: ')
try:
   val_n = int(my_n)
   val_m = int(my_m)
except ValueError:
    print("No.. input string is not an Integer. It's a string")
    sys.exit("program is aborted....")

member = my_solution_t1(val_m, val_n)

Please enter the range of values into the universe: 20
Please enter the num of elements that you want: 15


In [9]:
functions,buckets =member.get_functions()
for i in range(len(functions)):
   print("function is (",functions[i][0] , "* key + ", functions[i][1], ") module", buckets)

function is ( 21 * key +  14 ) module 23


In [10]:
import numpy as np
alphabet = {
            1:"A" , 2:"B", 3:"C", 4:"D" , 5:"E" , 6:"F" , 7:"G" , 8:"H", 9:"I", 0: "J"
     }
for i in range(0,int(val_m)):
    key = ""
    for j in range(1,4):
          l= np.random.randint(0,10)
          key = key + alphabet[l]
    pos, mapped_key = member.get_hash(key)
    member.set_item(pos)

In [11]:
print(member.find_item("HHF"))

True


In [12]:
print(member.find_item("DGF"))

False


In [13]:
print(member.load_factor())

0.5217391304347826


In [14]:
member._k

1

### IV. Submit your solution to the grader

In [20]:
## TEACHER
student.submit_task(globals(), task_id="T1");

## TASK: T2

<p justify="align">
<font color='0B5345' ></font><b>Bloom filter theory.</b></font>
<ol align="justify">
<li> Find step by step $P(one \ \   h_i(x) , (i...1...k), \ \ hit \ \ a \ \ given \ \ bucket)$ Having $n$ buckets, $m$ elements and $k$ hash functions.</li>
<li>¿How many functions do you need to minimize collisions, when $m= 10.000.000$ and  the available memory is 10 Megabytes? </li>
<li>You need a false positive rate  $p=0.01$,    when the value of $m$  increases to $100.000.000$, find the new value to $n$ to keep this false positive rate.</li>
<li>Suppose a set    S={  valid email addresses  }  and  $|S|=10^9$ . When an email address arrives in the stream, you need to verify if the account is new or existed previously.  Let suppose that we only have  available  one gigabyte of memory that is equivalent to keep a bit array of $8 *10^9$ buckets. Find the optimal number of hash functions to minimize the false positive rate. </li>
</ol>